In [2]:
from langchain.prompts import ChatPromptTemplate

# Prompt cho tác vụ nhận diện thực thể, quan hệ và thuộc tính
prompt_vietnam = (
	"# Hướng dẫn xây dựng Đồ thị Tri thức từ văn bản quy chế tiếng Việt\n"
	"## 1. Mục tiêu\n"
	"Bạn là một mô hình trích xuất thông tin chất lượng cao, được thiết kế để xây dựng đồ thị tri thức từ văn bản quy định, đặc biệt là quy chế đào tạo đại học bằng tiếng Việt.\n"
	"Hãy cố gắng trích xuất tối đa thông tin có trong văn bản **mà không thêm bất kỳ thông tin nào không được nêu rõ**.\n"
	"- **Node** là các thực thể hoặc khái niệm xuất hiện trong văn bản.\n"
	"- **Edge** là quan hệ giữa các node, phản ánh mối liên kết có trong nội dung văn bản.\n"
	"- Kết quả phải dễ hiểu, rõ ràng, có thể dùng để xây dựng hệ thống truy vấn đồ thị tri thức.\n\n"

	"## 2. Gán nhãn thực thể (Labeling Nodes)\n"
	"- Mỗi node phải có `id` là tên thực thể trích từ văn bản, và `type` là loại thực thể.\n"
	"- Nếu thực thể thuộc các loại thường gặp như: `CHƯƠNG`, `ĐIỀU`, `KHOẢN`, `CHỦ_THỂ`, `HOẠT_ĐỘNG`, `YÊU_CẦU`, `CÓ`, `LÀ`, `HỌC_LỰC`, `KHÁI_NIỆM`, ... thì hãy dùng đúng loại đó.\n"
	"- Nếu thực thể không thuộc các loại trên, HÃY TỰ ĐẶT `type` mới phù hợp bằng tiếng Việt.\n"
	"- Không được dùng số làm `id`. `id` phải là tên nguyên văn hoặc dễ hiểu từ văn bản, ví dụ: \"Chương trình đào tạo\", \"Sinh viên\".\n"
	"- Nếu trong văn bản có mô tả hoặc đặc điểm đi kèm thực thể, hãy trích xuất dưới dạng `properties`.\n"
	"- Mỗi node có thể có thêm các thuộc tính khác như `tiêu_đề`, `tên`, `định_nghĩa`, `mô_tả`, `vai_trò`, `phạm_vi`, v.v.\n"
	"- Nếu thuộc tính không nằm trong danh sách gợi ý, bạn được phép tự đặt tên phù hợp (vẫn theo chuẩn viết thường bằng tiếng Việt có dấu gạch dưới).\n\n"

	"## 3. Gán nhãn quan hệ (Relationships)\n"
	"- Các `edges` phải mô tả chính xác mối liên hệ giữa thực thể nguồn (`source`) và thực thể đích (`target`).\n"
	"- `type` của edge phải là một quan hệ rõ ràng, ngắn gọn, bằng **tiếng Việt viết HOA có dấu gạch dưới**, ví dụ: `QUY_ĐỊNH`, `BAO_GỒM`, `PHẢI_HOÀN_THÀNH`, `LIÊN_QUAN_ĐẾN`, `XẾP_LOẠI`, `THỜI_GIAN`, `LÀM`, `THUỘC`, `THAM_CHIẾU`.\n"
	"- Không dùng tên quan hệ quá cụ thể theo ngữ cảnh tạm thời, ví dụ: `BỊ_CẢNH_BÁO_2_LẦN` là không hợp lệ. Hãy dùng chung như `BỊ_CẢNH_BÁO`.\n"
	"- Nếu quan hệ không nằm trong danh sách gợi ý, bạn được phép tự đặt tên phù hợp (vẫn theo chuẩn viết HOA bằng tiếng Việt có dấu gạch dưới).\n"
	"- Nếu có thông tin bổ sung về quan hệ (ví dụ: điều_kiện, quy_định_chi_tiết, v.v.), hãy thêm vào `properties` của quan hệ.\n"
	"- Nếu thuộc tính không nằm trong danh sách gợi ý, bạn được phép tự đặt tên phù hợp (vẫn theo chuẩn viết thường bằng tiếng Việt có dấu gạch dưới).\n\n"

	"## 4. Đồng nhất thực thể (Coreference Resolution)\n"
	"- Nếu một thực thể xuất hiện nhiều lần với cách gọi khác nhau (VD: \"Trường\", \"Nhà Trường\", \"Trường Đại học Quy Nhơn\", \"trường\"), hãy dùng tên đầy đủ nhất làm ID duy nhất.\n"
	"- Không tạo trùng node cho cùng một thực thể.\n\n"

	"## 5. Tuân thủ định dạng và chính xác\n"
	"- Không giải thích, không thêm bình luận.\n"
	"- Chỉ trả về kết quả theo định dạng JSON gồm `nodes` và `edges`.\n"
	"- Kết quả phải đầy đủ, rõ ràng và có thể phân tích được bằng máy.\n"
)

new_prompt = ChatPromptTemplate.from_messages(
	[
		(
			"system",
			prompt_vietnam,
		),
		(
			"human",
			(
				"Tip: Make sure to answer in the correct format and do "
				"not include any explanations. "
				"Use the given format to extract information from the "
				"following input: {input}"
			),
		),
	]
)

In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
load_dotenv()

# Mô hình gpt-4o
os.environ["OPENAI_API_KEY"] = str(os.getenv("OPENAI_API_KEY")) # API OPENAI
llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [4]:
llm_transformer = LLMGraphTransformer(llm=llm, prompt=new_prompt, node_properties=True ,relationship_properties=True)
# llm_transformer = LLMGraphTransformer(llm=llm, node_properties=True ,relationship_properties=True)

In [14]:
arr = ['''Tên quy định: Quy định chuẩn đầu ra tin học đối với sinh viên đại học hệ chính quy
Điều 1. Phạm vi và đối tượng áp dụng
1. Văn bản này quy định Chuẩn đầu ra tin học đối với sinh viên đại học hệ chính quy Trường Đại học Quy Nhơn
2. Quy định này áp dụng từ khóa tuyển sinh năm 2015 (khóa 38) trở đi.
3. Quy định này không áp dụng đối với sinh viên ngành Sư phạm Tin học, Công nghệ thông tin.
Điều 2. Chuẩn đầu ra tin học
1. Chuẩn đầu ra tin học được xác định theo Chuẩn kỹ năng sử dụng công nghệ thông tin quy định tại Thông tư số 03/2014/TT-BTTTT ngày 11/03/2014 của Bộ Thông tin và Truyền thông.
2. Chuẩn đầu ra tin học là điều kiện để xét công nhận tốt nghiệp.
3. Trước khi xét tốt nghiệp, sinh viên phải đạt một trong các chứng chỉ sau đây:
a) Chứng chỉ ứng dụng công nghệ thông tin cơ bản do Trường Đại học Quy Nhơn cấp.
b) Chứng chỉ tin học quốc tế IC3 (Internet and Computing Core Certification).
c) Chứng chỉ tin học quốc tế ICDL (International Computer Driving Licence) - đạt ít nhất 6 mô đun sau: ICDL.01, ICDL.02, ICDL.03, ICDL.04, ICDL.05, ICDL.08.
d) Chứng chỉ tin học quốc tế MOS (Microsoft Office Specialist) - đạt 3 chứng chỉ Word/Excel/PowerPoint.
4. Các trường hợp đặc biệt do Hiệu trưởng quyết định.
Điều 3. Tổ chức thực hiện
1. Các ông (bà) Trưởng phòng Khảo thí và Đảm bảo chất lượng, Đào tạo đại học, Công tác sinh viên và các Trưởng đơn vị có liên quan chịu trách nhiệm triển khai và hướng dẫn thực hiện Quy định này.
2. Trung tâm tin học Trường Đại học Quy Nhơn phối hợp với các đơn vị liên quan tổ chức thi và cấp chứng chỉ Chuẩn đầu ra tin học cho sinh viên.
Trong quá trình thực hiện, nếu có vấn đề chưa phù hợp, các đơn vị đề xuất với Nhà trường (qua Phòng Khảo thí và Đảm bảo chất lượng) để bổ sung, điều chỉnh.''']


In [ ]:
# Đọc hết tất cả 14 file để chuẩn bị chuyển thành đồ thị
base_dir = os.getcwd()

# Tạo đường dẫn đến thư mục txt
folder_path = os.path.join(base_dir, 'app', 'static', 'txt')
folder_path = 'D:/WorkSpace/python/thesis/web_graph_rag/app/static/txt'
for filename in os.listdir(folder_path):
	if filename.endswith('.txt'):
		file_path = os.path.join(folder_path, filename)
		with open(file_path, 'r', encoding='utf-8') as f:
			content = f.read()
			arr = arr + content.split('\n\n')
			# print(f"Nội dung của {filename}:\n{content}\n")

# print(len(arr))

In [5]:
# Đọc 1 file
base_dir = os.getcwd()
# Quy chế đào tạo trình độ đại học.txt
# Quy định về học phí của sinh viên đại học hệ chính quy.txt
# dữ liệu đánh giá.txt
arr = []
file_name = 'miss.txt'
# folder_path = os.path.join(base_dir, 'app', 'static', 'txt', file_name)
folder_path = os.path.join(base_dir, file_name)
with open(folder_path, 'r', encoding='utf-8') as f:
	content = f.read()
	arr = arr + content.split('\n\n')
	# arr.append(content)
	
len(arr)

5

In [6]:
# Tạo cấu trúc Document để vào LLMGraphTransformer xử lý
documents = []
for x in arr:
	documents.append(Document(page_content=x))

In [7]:
# Chuyển các document thành các thực thể và quan hệ
graph_documents = llm_transformer.convert_to_graph_documents(documents=documents)

In [8]:
# Hiển thị các thực thể và quan hệ
for x in graph_documents:
	print(f"Nodes:{x.nodes}")
	print(f"Relationships:{x.relationships}")
	print()

Nodes:[Node(id='Ngành Sư Phạm Tiếng Anh', type='Ngành', properties={}), Node(id='Ngành Ngôn Ngữ Anh', type='Ngành', properties={}), Node(id='Ngành Ngôn Ngữ Trung Quốc', type='Ngành', properties={}), Node(id='Ngành Chất Lượng Cao', type='Ngành', properties={}), Node(id='Các Ngành Khác', type='Ngành', properties={}), Node(id='Trường Đại Học Quy Nhơn', type='Tổ_chức', properties={}), Node(id='Bộ Giáo Dục Và Đào Tạo', type='Tổ_chức', properties={}), Node(id='Ngoại Ngữ 1', type='Khái_niệm', properties={}), Node(id='Ngoại Ngữ 2', type='Khái_niệm', properties={}), Node(id='Tiếng Anh Bậc 5/6', type='Yêu_cầu', properties={}), Node(id='Tiếng Pháp Bậc 3/6', type='Yêu_cầu', properties={}), Node(id='Tiếng Trung Bậc 3/6', type='Yêu_cầu', properties={}), Node(id='Tiếng Anh Bậc 3/6', type='Yêu_cầu', properties={}), Node(id='Tiếng Pháp Bậc 4/6', type='Yêu_cầu', properties={}), Node(id='Tiếng Anh Bậc 4/6', type='Yêu_cầu', properties={}), Node(id='Cefr C1', type='Chứng_chỉ', properties={}), Node(id='Cae 

In [8]:
# Một các hiển thị dễ quan sát hơn
for x in graph_documents:
	for y in x.nodes:
		print(f"{y.type} - {y.id} - {y.properties}")
	print()
	for y in x.relationships:
		print(f"({y.source.type}: {y.source.id}) - ({y.type}:{y.properties}) - ({y.target.type}:{y.target.id})")
	
	print('-'*40)
	print()

Ngành - Ngành Sư Phạm Tiếng Anh (Ngoại Ngữ 1) - {}
Ngành - Ngành Sư Phạm Tiếng Anh (Ngoại Ngữ 2) - {}
Ngành - Ngành Ngôn Ngữ Anh - {}
Ngành - Ngành Ngôn Ngữ Trung Quốc - {}
Ngành - Các Ngành Chất Lượng Cao - {}
Ngành - Các Ngành Khác - {}
Tổ_chức - Trường Đại Học Quy Nhơn - {}
Tổ_chức - Bộ Giáo Dục Và Đào Tạo - {}
Chứng_nhận - Chứng Nhận Tiếng Anh Bậc 5/6 - {}
Chứng_nhận - Chứng Nhận Tiếng Pháp Bậc 3/6 - {}
Chứng_nhận - Chứng Nhận Tiếng Trung Bậc 3/6 - {}
Chứng_nhận - Chứng Nhận Tiếng Anh Bậc 3/6 - {}
Chứng_nhận - Chứng Nhận Tiếng Pháp Bậc 4/6 - {}
Chứng_nhận - Chứng Nhận Tiếng Trung Bậc 4/6 - {}
Chứng_chỉ - Chứng Chỉ Quốc Tế - {'danh_sách': 'CEFR C1, CAE 60, IELTS 6.5, TOEFL iBT 85, TOEIC 850, APTIS ESOL GENERAL C, APTIS ESOL ADVANCED C1'}
Chứng_chỉ - Chứng Chỉ Quốc Tế - {'danh_sách': 'JLPT N4, JTEST Cấp độ E, HSK Cấp độ 3, DELF B1, TCF Niveau 3, TOPIK 3'}
Chứng_chỉ - Chứng Chỉ Quốc Tế - {'danh_sách': 'CEFR B1, IELTS 4.5, TOEFL iBT 45, TOEIC 450, APTIS ESOL GENERAL B1, APTIS ESOL ADVA

In [9]:
# Hàm để chuyển các thực thể và quan hệ vào neo4j
def insert_nodes_and_relationships(driver, nodes, relationships):
	with driver.session() as session:
		# Chèn nodes
		for node in nodes:
			x = str(node.type).replace(',', '')
			x = x.replace(' ', '_')

			cypher_query = f"""
				MERGE (n:{x} {{id: $id}})
				SET n += $properties
			"""
			session.run(
				cypher_query,
				id=node.id,
				properties=node.properties
			)

		# Chèn relationships
		for relationship in relationships:
			x = str(relationship.source.type).replace(',', '')
			x = x.replace(' ', '_')

			y = str(relationship.target.type).replace(',', '')
			y = y.replace(' ', '_')

			z = str(relationship.type).replace(',', '')
			z = z.replace(' ', '_')

			cypher_query = f"""
				MATCH (a:{x} {{id: $source_id}}), 
							(b:{y} {{id: $target_id}})
				MERGE (a)-[r:{z}]->(b)
				SET r += $properties
			"""	
			session.run(
				cypher_query,
				source_id=relationship.source.id,
				target_id=relationship.target.id,
				properties=relationship.properties
			)

In [10]:
# Hàm tạo connect đến neo4j
from neo4j import GraphDatabase

def connect_to_neo4j(uri, user, password, db):
	driver = GraphDatabase.driver(uri, auth=(user, password), database=db)
	return driver

In [11]:
# Kết nối đến Neo4j
uri = "bolt://localhost:7687"
user = "neo4j"
password = "adminadmin"
db = 'quydinh-quyche-full-temperature-0'
driver = connect_to_neo4j(uri, user, password, db)

In [12]:
for x in graph_documents:
	insert_nodes_and_relationships(driver, x.nodes, x.relationships)


In [57]:
driver.close()


In [37]:
# Prompt cho tác vụ tìm kiếm(cũ)
CYPHER_GENERATION_TEMPLATE = """
Task: Sinh truy vấn Cypher để truy xuất dữ liệu từ đồ thị tri thức về quy chế đào tạo đại học.

Instructions:
- Phân tích kỹ câu hỏi và trích xuất các thành phần quan trọng như thực thể (node), quan hệ (edge) và thuộc tính (property).
- Chỉ sử dụng các loại node, quan hệ và thuộc tính có trong schema bên dưới. Không thêm bất kỳ phần tử nào không có trong schema.
- Các nhãn node được đặt bằng tiếng Việt (ví dụ: :Chương_trình, :Chủ_thể, :Học, :Quy_định...).
- Các nhãn quan hệ được đặt bằng tiếng Việt (ví dụ: :ÁP_DỤNG, :HỌC, :QUY_ĐỊNH...).
- Truy vấn phải trả lời chính xác câu hỏi dựa trên cấu trúc đồ thị.
- Chỉ trả về truy vấn Cypher. Không viết giải thích, ghi chú, hoặc định nghĩa gì thêm.

Schema đồ thị:
{schema}

Ví dụ truy vấn:

# Các phương thức đóng học phí bao gồm các phương thức nào?
# Cách 1
MATCH (n:`Hoạt_động`)-[]->(m)
WHERE n.id contains 'Phương Thức Đóng Học Phí'
return n, m
# Cách 2
MATCH (n:`Phương_thức`)-[:`ÁP_DỤNG_CHO`]->(m)
WHERE m.id contains 'Sinh Viên'
return n.id

# Học phần bắt buộc là gì?
MATCH (n: Loại_học_phần)
WHERE n.id contains 'Học Phần Bắt Buộc'
return n.`định_nghĩa`

# Khi nào sinh viên bị buộc thôi học?
MATCH (n: Chủ_thể)-[:`CÓ_HÀNH_VI`]->(m)-[:`BỊ_XỬ_LÝ`]->(x)
WHERE n.id contains 'Sinh Viên' 
AND x.id contains 'Buộc Thôi Học'
return n, m, x

# Thời hạn đóng học phí là khi nào?
MATCH (n:`Chủ_thể`)-[r1:`PHẢI_TUÂN_THỦ`]->(m)-[r2]->(x)
where n.id contains 'Sinh Viên' 
and m.id contains 'Thời Hạn Đóng Học Phí'
RETURN n, r1, m, r2, x 

The question is:
{question}
"""


In [42]:
# Prompt cho tác vụ tìm kiếm(mới)

CYPHER_GENERATION_TEMPLATE = """
Task: Sinh truy vấn Cypher để truy xuất dữ liệu từ đồ thị tri thức về quy chế đào tạo đại học.

Instructions:
- Phân tích kỹ câu hỏi và xác định các thành phần quan trọng như thực thể (node), quan hệ (edge), và thuộc tính (property).
- Chỉ sử dụng các loại node, quan hệ và thuộc tính có trong schema bên dưới. Không thêm bất kỳ phần tử nào không có trong schema.
- Các nhãn node được đặt bằng tiếng Việt (ví dụ: :Chương_trình, :Chủ_thể, :Học, :Quy_định...).
- Các loại quan hệ cũng được đặt bằng tiếng Việt (ví dụ: :ÁP_DỤNG, :HỌC, :QUY_ĐỊNH...).
- Mọi truy vấn **phải trả về đầy đủ tất cả thuộc tính** của node và relationship bằng `properties(...)`. Nếu cần, sử dụng `type(...)` để lấy tên quan hệ.
- **Chỉ được sử dụng phép so sánh `CONTAINS` trong mệnh đề WHERE. Không được sử dụng phép gán bằng (`=`).**
- Không viết lời giải thích, ghi chú hoặc định nghĩa. Chỉ trả về truy vấn Cypher.

Schema đồ thị:
{schema}

Ví dụ truy vấn:

# Các phương thức đóng học phí bao gồm các phương thức nào?
MATCH (n:`Hoạt_động`)-[r]->(m)
WHERE n.id CONTAINS 'Phương Thức Đóng Học Phí'
RETURN 
  properties(n) AS n_properties, 
  type(r) AS r_type, 
  properties(r) AS r_properties, 
  properties(m) AS m_properties

# Cách các phương thức đóng học phí áp dụng cho sinh viên?
MATCH (n:`Phương_thức`)-[r:`ÁP_DỤNG_CHO`]->(m)
WHERE m.id CONTAINS 'Sinh Viên'
RETURN 
  properties(n) AS n_properties, 
  type(r) AS r_type, 
  properties(r) AS r_properties, 
  properties(m) AS m_properties

# Học phần bắt buộc là gì?
MATCH (n:Loại_học_phần)
WHERE n.id CONTAINS 'Học Phần Bắt Buộc'
RETURN properties(n) AS n_properties

# Khi nào sinh viên bị buộc thôi học?
MATCH (n:`Chủ_thể`)-[r1:`CÓ_HÀNH_VI`]->(m)-[r2:`BỊ_XỬ_LÝ`]->(x)
WHERE n.id CONTAINS 'Sinh Viên' 
  AND x.id CONTAINS 'Buộc Thôi Học'
RETURN 
  properties(n) AS n_properties, 
  type(r1) AS r1_type, 
  properties(r1) AS r1_properties,
  properties(m) AS m_properties, 
  type(r2) AS r2_type, 
  properties(r2) AS r2_properties, 
  properties(x) AS x_properties

# Thời hạn đóng học phí là khi nào?
MATCH (n:`Chủ_thể`)-[r1:`PHẢI_TUÂN_THỦ`]->(m)-[r2]->(x)
WHERE n.id CONTAINS 'Sinh Viên' 
  AND m.id CONTAINS 'Thời Hạn Đóng Học Phí'
RETURN 
  properties(n) AS n_properties, 
  type(r1) AS r1_type, 
  properties(r1) AS r1_properties, 
  properties(m) AS m_properties, 
  type(r2) AS r2_type, 
  properties(r2) AS r2_properties, 
  properties(x) AS x_properties

The question is:
{question}
"""


In [43]:
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_PROMPT = PromptTemplate(
	input_variables=["schema", "question"],
	template=CYPHER_GENERATION_TEMPLATE
)

In [45]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
load_dotenv()

graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="adminadmin", database="prompt-1-temperature-1", enhanced_schema=True)

# print(graph.schema)

os.environ["OPENAI_API_KEY"] = str(os.getenv("OPENAI_API_KEY"))
llm = ChatOpenAI(temperature=1, model="gpt-4o")

chain = GraphCypherQAChain.from_llm(
	llm,
	graph=graph,
	verbose=True,
	top_k=5,
	allow_dangerous_requests=True,
	# validate_cypher=True,
	cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [49]:
result = chain.invoke({"query": "Điều kiện nào cần để tham gia khóa luận tốt nghiệp?"})

print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:`Khái_niệm`)-[r]->(m)
WHERE n.id CONTAINS 'Khóa Luận Tốt Nghiệp'
RETURN 
  properties(n) AS n_properties, 
  type(r) AS r_type, 
  properties(r) AS r_properties, 
  properties(m) AS m_properties
Full Context:
[]

> Finished chain.
Final answer: Hiện tại tôi không có thông tin cụ thể về điều kiện tham gia khóa luận tốt nghiệp. Bạn vui lòng liên hệ trực tiếp với phòng đào tạo hoặc khoa chuyên môn của trường để được hướng dẫn chi tiết nhé.


In [18]:
# Cách hiển thị Final answer dễ quan sát hơn
import textwrap

def prettyCypherChain(question: str) -> str:
	response = chain.run(question)
	print(textwrap.fill(response, 60))

In [22]:
prettyCypherChain("Đào tạo chính quy là gì?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:Khái_niệm {id: "Đào Tạo Chính Quy"}) RETURN n.định_nghĩa

Full Context:
[{'n.định_nghĩa': None}]

> Finished chain.
Tôi không biết câu trả lời cho câu hỏi "Đào tạo chính quy là
gì?".
